pytorch安装

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

查询package里的函数以及函数如何使用

In [11]:
dir(torch.cuda)

['Any',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BoolStorage',
 'BoolTensor',
 'ByteStorage',
 'ByteTensor',
 'CUDAGraph',
 'CUDAPluggableAllocator',
 'Callable',
 'CharStorage',
 'CharTensor',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'CudaError',
 'DeferredCudaCallError',
 'Device',
 'DoubleStorage',
 'DoubleTensor',
 'Event',
 'ExternalStream',
 'FloatStorage',
 'FloatTensor',
 'HalfStorage',
 'HalfTensor',
 'IntStorage',
 'IntTensor',
 'List',
 'LongStorage',
 'LongTensor',
 'MemPool',
 'MemPoolContext',
 'Optional',
 'OutOfMemoryError',
 'ShortStorage',
 'ShortTensor',
 'Stream',
 'StreamContext',
 'Tuple',
 'Union',
 '_CudaBase',
 '_CudaDeviceProperties',
 '_DeviceGuard',
 '_HAS_PYNVML',
 '_LazySeedTracker',
 '_PYNVML_ERR',
 '_WrappedTritonKernel',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_cached_device_count',
 '_check_bf16_tensor_supported',
 '_check_c

In [12]:
#查函数不要加括号
help(torch.cuda.is_available)

Help on function is_available in module torch.cuda:

is_available() -> bool
    Return a bool indicating if CUDA is currently available.



### 读取数据:dataset与dataloader

dataset:提供一种方式获取数据及其label

dataloader：为后面的网络提供不同的数据形式

In [15]:
from torch.utils.data import Dataset
from PIL import Image
import os

In [ ]:
#自定义Dataset类用于读取图片数据
class MyDataset(Dataset):
    def __init__(self,root_dir,label_dir):
        self.root_dir = root_dir #数据集根目录
        self.label_dir = label_dir #标签目录 ants or bees
        self.path = os.path.join(self.root_dir,self.label_dir) #拼接路径
        self.img_path = os.listdir(self.path) #读取路径下的所有文件名称
    
    def __getitem__(self,idx):
        img_name = self.img_path[idx] #获取图片名称
        img_path = os.path.join(self.path,img_name) #拼接图片路径
        img = Image.open(img_path) #读取图片
        label = self.label_dir
        return img,label
    def __len__(self):
        return len(self.img_path) 

In [ ]:
#读取蚂蚁数据集
root_dir = r'F:\RUC\pytorch\数据集\hymenoptera_data\train'
label_dir = 'ants'
ants_dataset = MyDataset(root_dir,label_dir)

In [25]:
#打开图片
img,label = ants_dataset.__getitem__(0)
#或者使用:
#img,label = ants_dataset[0]
img.show()

#查看数据集大小
ants_dataset.__len__()
#或者使用:
#len(ants_dataset)

124

In [26]:
root_dir = r'F:\RUC\pytorch\数据集\hymenoptera_data\train'
label_dir = 'bees'
bees_dataset = MyDataset(root_dir,label_dir)

In [27]:
#合并数据集
train_dataset = ants_dataset + bees_dataset

In [28]:
#查看合并数数据集的信息
img,label = train_dataset.__getitem__(125)
img.show()

In [ ]:
#如果是图片+标签的形式 如何读取数据
class MyDataset2(Dataset):
    def __init__(self,root_dir,image_dir,label_dir):
        self.root_dir = root_dir #数据集根目录
        self.image_dir = image_dir #图片目录
        self.label_dir = label_dir #标签目录
        self.image_path = os.path.join(self.root_dir,self.image_dir) #拼接路径并读取
        self.label_path = os.path.join(self.root_dir,self.label_dir)
        self.image_list = os.listdir(self.image_path)
        self.label_list = os.listdir(self.label_path)
        # 因为label 和 Image文件名相同，进行一样的排序，可以保证取出的数据和label是一一对应的
        self.image_list.sort()
        self.label_list.sort()
 
    def __getitem__(self,idx):
        #图片、标签名称/路径
        img_name = self.image_list[idx]
        label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        #读取图片、标签
        img = Image.open(img_item_path)

        with open(label_item_path, 'r') as f:
            label = f.readline()

        # img = np.array(img)
        #?这里为什么要转换为tensor
        #img = self.transform(img) 暂时先不用transform
        sample = {'img': img, 'label': label}
        return sample

    def __len__(self):
        return len(self.label_list) 

In [39]:
root_dir = r'F:\RUC\pytorch\数据集\练手数据集\train'
image_dir = 'ants_image'
label_dir = 'ants_label'

ants_dataset = MyDataset2(root_dir,image_dir,label_dir)

In [42]:
#打开图片
ants_dataset[0]['img'].show()

### Tensorboard的使用

SummaryWriter类是 PyTorch 中 torch.utils.tensorboard 模块提供的一个重要工具，主要用于将训练过程中的各种数据（如损失值、准确率、图像等）写入 TensorBoard 可以读取的日志文件，方便用户通过 TensorBoard 可视化工具直观地观察和分析模型的训练过程和性能。

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [7]:
#创建类
writer = SummaryWriter('logs')

for i in range(100):
    writer.add_scalar('y = 2x',2*i,i)
#关闭类
writer.close()

In [10]:
from PIL import Image
import numpy as np

#打开一张图片然后转成numpy类型
image_path = r'F:\RUC\pytorch\数据集\练手数据集\train\ants_image\0013035.jpg'
image_PIL = Image.open(image_path)
image_array = np.array(image_PIL)
#查看array的通道数
print(image_array.shape)

#创建类
writer = SummaryWriter('logs')
#转换成numpy数组后需要修改dataformats,因为和默认的不一样
writer.add_image('test',image_array,1,dataformats='HWC')
writer.close()

(512, 768, 3)


### Transforms

In [11]:
from torchvision import transforms

In [12]:
from PIL import Image

In [14]:
#读取一个图片
img_path = r'F:\RUC\pytorch\数据集\练手数据集\train\ants_image\541630764_dbd285d63c.jpg'
img = Image.open(img_path)

#定义totensor对象 转换图片
tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)

#查看tensor
tensor_img

tensor([[[0.3608, 0.3686, 0.3686,  ..., 0.4039, 0.4000, 0.4078],
         [0.3569, 0.3647, 0.3686,  ..., 0.4078, 0.4078, 0.4157],
         [0.3686, 0.3608, 0.3569,  ..., 0.4039, 0.4118, 0.4157],
         ...,
         [0.3725, 0.3686, 0.3686,  ..., 0.8902, 0.8863, 0.8824],
         [0.3608, 0.3608, 0.3647,  ..., 0.8941, 0.8902, 0.8902],
         [0.3608, 0.3608, 0.3647,  ..., 0.8941, 0.8902, 0.8863]],

        [[0.5686, 0.5725, 0.5725,  ..., 0.6235, 0.6196, 0.6275],
         [0.5647, 0.5686, 0.5725,  ..., 0.6275, 0.6275, 0.6353],
         [0.5765, 0.5647, 0.5608,  ..., 0.6314, 0.6392, 0.6431],
         ...,
         [0.5922, 0.5882, 0.5843,  ..., 0.9176, 0.9137, 0.9098],
         [0.5765, 0.5765, 0.5843,  ..., 0.9137, 0.9098, 0.9098],
         [0.5765, 0.5765, 0.5843,  ..., 0.9137, 0.9098, 0.9059]],

        [[0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0039, 0.0039,  ..., 0.0039, 0.0000, 0.0039],
         [0.0118, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.

In [ ]:
#常见的transforms

#打开一张图片
from PIL import Image
from torchvision import transforms
img = Image.open(r'F:\RUC\pytorch\数据集\练手数据集\train\ants_image\2288481644_83ff7e4572.jpg')
                 
#compose
#transforms.Compose将多个transforms方法组合起来使用
#比如将图片先resize到256*256，然后随机裁剪到224*224，最后转换为tensor


#totensor
#将PIL Image或者 ndarray 转换为tensor，并且归一化到[0-1.0]之间
tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)


In [16]:
#normalize
#将每个信道的数据标准化到设定的均值和标准差
#标准化前
print(tensor_img[0][0][0])
trans_norm = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
img_norm = trans_norm(tensor_img)
#标准化后
print(img_norm[0][0][0])

tensor(0.2784)
tensor(-0.4431)


In [18]:
#resize
#调整图片大小
print(img.size)
trans_resize = transforms.Resize((256,256))
img_resize = trans_resize(img)
print(img_resize.size)
#如果要在tensorboard中显示，需要转换成tensor  

(500, 500)
(256, 256)


In [ ]:
#randomcrop
#随机裁剪图片
trans_random = transforms.RandomCrop(256)
trans_compose = transforms.Compose([trans_random,tensor_trans])
for i in range(10):
    img_crop = trans_compose(img)
    writer.add_image('randomcrop',img_crop,i)
#在tensorboard中查看各步骤结果
writer.close()


In [22]:
#下载torchvision中的数据集且不进行transform操作
import torchvision
train_set = torchvision.datasets.CIFAR10(root='./data',train=True,download=True)
test_set = torchvision.datasets.CIFAR10(root='./data',train=False,download=True)

#查看原始数据集
img,target = train_set[0]
img.show()
#查看标签
print(train_set.classes[target])

frog


In [ ]:
#下载torchvision中的数据集且进行transform操作
#用compose定义transform
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_set = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

### Dataloader

In [ ]:
#从torchvision中加载数据集
from torch.utils.data import DataLoader
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
test_set = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

test_loader = DataLoader(test_set,batch_size=64,shuffle=True,num_workers=0,drop_last=True)
#batch_size:每次读取并合并的数据量,然后将所有数据按照该方式划分成n//4批

#查看原始数据集的第一张图片及标签
img,target = test_set[0]
print(img.shape)
print(target)

#查看dataloader中的数据
writer = SummaryWriter('dataloader')
step = 0
#epoch是指遍历整个数据集的次数
for epoch in range(2):
    for data in test_loader:
        img,target = data
        #print(img.shape)
        #print(target)
        writer.add_images('test_drop',img,step)
        step += 1
writer.close()

torch.Size([3, 32, 32])
3
